In [ ]:
 from google.colab import drive
drive.mount('/content/drive/',force_remount=False)

Mounted at /content/drive/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd "/content/drive/My Drive/Attacks"

/content/drive/My Drive/Attacks


In [ ]:
!ls

 attackDataset		       Labelsbwg.csv
 attack.py		       LabelsLH.csv
 averageattack.csv	       LabelsR.csv
 AverageAttack.csv	       LabelsRD.csv
 averageAttack.py	       LH.csv
 avg.csv		       LoveHateattack.csv
 bandattack.csv		       Mainattack.ipynb
 band.csv		       main.py
 bandwagonAttack.py	       MovieAttacks.ipynb
 config			       __pycache__
 config.py		       Randomattack_10_10.csv
 dataset		       randomattack.csv
 Ddataset		       Randomattack.csv
'DdatasetCoDetector@Time[1]'   RandomAttack.csv
'DdatasetCoDetector@Time[2]'   randomAttack.py
'DdatasetCoDetector@Time[3]'   ratings.csv
'DdatasetCoDetector@Time[4]'   R.csv
'DdatasetCoDetector@Time[5]'   RD.csv
 detectiondataset	       RMSE3
 final_avgmain.csv	      'Rmse bar'
 finallh.csv		      ' SD'
 finalLH.csv		      ' SDLib'
 final_main.csv		      ' SDLibMR_Avg_A_5_F_0.03.csv'
 final_random.csv	       shillingfile.py
 final_rmain.csv	       tavg.csv
 finalrnd.csv		       tbwg.csv
 GANattack.py		       tLH.csv
 getTarget

# Warming up

In [ ]:
# config, line config , file IO , Data split ,RatingDAO
import os.path
from os.path import abspath
class Config(object):
    def __init__(self,fileName):
        self.config = {}
        self.readConfiguration(fileName)

    def __getitem__(self, item):
        if not self.contains(item):
            print('parameter ')+item+(' is invalid!')
            exit(-1)
        return self.config[item]

    def getOptions(self,item):
        if not self.contains(item):
            print('parameter ')+item+(' is invalid!')
            exit(-1)
        return self.config[item]

    def contains(self,key):
        return key in self.config 

    def readConfiguration(self,fileName):
        if not os.path.exists(abspath(fileName)):
            print('config file is not found!')
            raise IOError
        with open(fileName) as f:
            for ind,line in enumerate(f):
                if line.strip()!='':
                    try:
                        key,value=line.strip().split('=')
                        self.config[key]=value
                    except ValueError:
                        print('config file is not in the correct format! Error Line:%d')%(ind)


class LineConfig(object):
    def __init__(self,content):
        self.line = content.strip().split(' ')
        self.options = {}
        self.mainOption = False
        if self.line[0] == 'on':
            self.mainOption = True
        elif self.line[0] == 'off':
            self.mainOption = False
        for i,item in enumerate(self.line):
            if (item.startswith('-') or item.startswith('--')) and  not item[1:].isdigit():
                ind = i+1
                for j,sub in enumerate(self.line[ind:]):
                    if (sub.startswith('-') or sub.startswith('--')) and  not sub[1:].isdigit():
                        ind = j
                        break
                    if j == len(self.line[ind:])-1:
                        ind=j+1
                        break
                try:
                    self.options[item] = ' '.join(self.line[i+1:i+1+ind])
                except IndexError:
                    self.options[item] = 1

    def __getitem__(self, item):
        if not self.contains(item):
            print ('parameter ')+item+(' is invalid!')
            exit(-1)
        return self.options[item]

    def getOption(self,key):
        if not self.contains(key):
            print ('parameter ')+key+(' is invalid!')
            exit(-1)
        return self.options[key]

    def isMainOn(self):
        return self.mainOption

    def contains(self,key):
        return key in self.options


In [ ]:
import os.path
from os import makedirs,remove
from re import compile,findall,split
from collections import defaultdict
class FileIO(object):
    def __init__(self):
        pass


    @staticmethod
    def writeFile(dir,file,content,op = 'w'):
        if not os.path.exists(dir):
            os.makedirs(dir)
        if type(content)=='str':
            with open(dir + file, op) as f:
                f.write(content)
        else:
            with open(dir+file,op) as f:
                f.writelines(content)

    @staticmethod
    def deleteFile(filePath):
        if os.path.exists(filePath):
            remove(filePath)

    @staticmethod
    def loadDataSet(conf, file, bTest=False):
        trainingData = defaultdict(dict)
        testData = defaultdict(dict)
        ratingConfig = LineConfig(conf['ratings.setup'])
        if not bTest:
            print('loading training data...')
        else:
            print('loading test data...')
        with open(file) as f:
            ratings = f.readlines()
        # ignore the headline
        if ratingConfig.contains('-header'):
            ratings = ratings[1:]
        # order of the columns
        order = ratingConfig['-columns'].strip().split()
        # count =0

        for lineNo, line in enumerate(ratings):
            items = split(' |,|\t', line.strip())
            if not bTest and len(order) < 3:
                print('The rating file is not in a correct format. Error: Line num %d')% lineNo
                exit(-1)
            try:
                userId = items[int(order[0])]
                itemId = items[int(order[1])]
                if bTest and len(order)<3:
                    rating = 1 #default value
                else:
                    rating  = items[int(order[2])]

            except ValueError:
                print('Error! Have you added the option -header to the rating.setup?')
                exit(-1)
            # count+=1
            if not bTest:
                trainingData[userId][itemId]=float(rating)
                # print(f"c {count} {rating}")
            else:
                testData[userId][itemId] = float(rating)
                # print(f"c {count} {rating}")
        if not bTest:
            return trainingData
        else:
            return testData

    @staticmethod
    def loadLabels(filePath):
        labels = {}
        with open(filePath) as f:
            for line in f:
                # items = split(' |,|\t', line.strip())
                line = line.strip()
                items = line.split(',')
                # print(line)
                labels[items[0]] = items[1]
        return labels

In [ ]:
from random import random
#from tool.file import FileIO
class DataSplit(object): 
    def __init__(self):
        pass
    @staticmethod
    def dataSplit(data,test_ratio = 0.3,output=False,path='./',order=1):
        if test_ratio>=1 or test_ratio <=0:
            test_ratio = 0.3
        testSet = {}
        trainingSet = {}
        for user in data:
            if random() < test_ratio:
                testSet[user] = data[user].copy()
            else:
                trainingSet[user] = data[user].copy()
        if output:
            FileIO.writeFile(path,'testSet['+str(order)+']',testSet)
            FileIO.writeFile(path, 'trainingSet[' + str(order) + ']', trainingSet)
        return trainingSet,testSet

    @staticmethod
    def crossValidation(data,k,output=False,path='./',order=1):
        if k<=1 or k>10:
            k=3
        for i in range(k):
            trainingSet = {}
            testSet = {}
            for ind,user in enumerate(data):
                if ind%k == i:
                    testSet[user] = data[user].copy()
                else:
                    trainingSet[user] = data[user].copy()
            yield trainingSet,testSet

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances,cosine_similarity
import numpy as np
from numpy.linalg import norm
from scipy.stats.stats import pearsonr
from math import sqrt,exp

def normalize(vec,maxVal,minVal):
    'get the normalized value using min-max normalization' 
    if maxVal > minVal:
        return float(vec-minVal)/(maxVal-minVal)+0.01
    elif maxVal==minVal:
        return vec/maxVal
    else:
        print('error... maximum value is less than minimum value.')
        raise ArithmeticError

<ipython-input-9-da17c8d1b81d>:4: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [ ]:
import numpy as np
import os.path
from re import split
from collections import defaultdict

class RatingDAO(object):
    'data access control'
    def __init__(self,config, trainingData, testData):
        self.config = config
        self.ratingConfig = LineConfig(config['ratings.setup'])
        self.user = {} #used to store the order of users in the training set
        self.item = {} #used to store the order of items in the training set
        self.id2user = {}
        self.id2item = {}
        self.all_Item = {}
        self.all_User = {}
        self.userMeans = {} #used to store the mean values of users's ratings
        self.itemMeans = {} #used to store the mean values of items's ratings

        self.globalMean = 0
        self.timestamp = {}
        # self.trainingMatrix = None
        # self.validationMatrix = None
        self.testSet_u = testData.copy() # used to store the test set by hierarchy user:[item,rating]
        self.testSet_i = defaultdict(dict) # used to store the test set by hierarchy item:[user,rating]
        self.trainingSet_u = trainingData.copy()
        self.trainingSet_i = defaultdict(dict)

        self.trainingData = trainingData
        self.testData = testData
        self.__generateSet()
        self.__computeItemMean()
        self.__computeUserMean()
        self.__globalAverage()



    def __generateSet(self):
        scale = set()
        for i,user in enumerate(self.trainingData):
            for item in self.trainingData[user]:

                self.trainingSet_i[item][user] = self.trainingData[user][item]
                # order the user
                if not user in self.user:
                    self.user[user] = len(self.user)
                    self.id2user[self.user[user]] = user
                # order the item
                if not item in self.item:
                    self.item[item] = len(self.item)
                    self.id2item[self.item[item]] = item
                self.trainingSet_i[item][user] = self.trainingData[user][item]
                # userList.append

        self.all_User.update(self.user)
        self.all_Item.update(self.item)

        for i, user in enumerate(self.testData):
            # order the user
            #if not self.user.has_key(user):
            if not user in self.user:
                self.all_User[user] = len(self.all_User)
            for item in self.testData[user]:
                # order the item
                #if not self.item.has_key(item):
                if not item in self.item:
                    self.all_Item[item] = len(self.all_Item)
                #self.testSet_u[userName][itemName] = float(rating)
                self.testSet_i[item][user] = self.testData[user][item]

    def __globalAverage(self):
        total = sum(self.userMeans.values())
        if total==0:
            self.globalMean = 0
        else:
            self.globalMean = total/len(self.userMeans)

    def __computeUserMean(self):
        for u in self.trainingSet_u:
            self.userMeans[u] = sum(self.trainingSet_u[u].values())/(len(self.trainingSet_u[u].values())+0.0)
        for u in self.testSet_u:
            self.userMeans[u] = sum(self.testSet_u[u].values())/(len(self.testSet_u[u].values())+0.0)

    def __computeItemMean(self):
        for item in self.trainingSet_i:
            self.itemMeans[item] = sum(self.trainingSet_i[item].values())/(len(self.trainingSet_i[item].values()) + 0.0)
        for item in self.testSet_i:
            self.itemMeans[item] = sum(self.testSet_i[item].values())/(len(self.testSet_i[item].values()) + 0.0)

    def getUserId(self,u):
        if self.user.has_key(u):
            return self.user[u]
        else:
            return -1

    def getItemId(self,i):
        if self.item.has_key(i):
            return self.item[i]
        else:
            return -1

    def trainingSize(self):
        recordCount = 0
        for user in self.trainingData:
            recordCount+=len(self.trainingData[user])
        return (len(self.trainingSet_u),len(self.trainingSet_i),recordCount)

    def testSize(self):
        recordCount = 0
        for user in self.testData:
            recordCount += len(self.testData[user])
        return (len(self.testSet_u),len(self.testSet_i),recordCount)

    def contains(self,u,i):
        'whether user u rated item i'
        if self.trainingSet_u.has_key(u) and self.trainingSet_u[u].has_key(i):
            return True
        return False

    def containsUser(self,u):
        'whether user is in training set'
        return self.trainingSet_u.has_key(u)
        
    def containsItem(self,i):
        'whether item is in training set'
        return self.trainingSet_i.has_key(i)
       
    def allUserRated(self, u):
        if u in self.user:
            return self.trainingSet_u[u].keys(), self.trainingSet_u[u].values()
             
        else:
            return self.testSet_u[u].keys(), self.testSet_u[u].values()

# DETECTION


In [ ]:
#from baseclass.SDetection import SDetection        (1)
class SDetection(object):
    def __init__(self,conf,trainingSet=None,testSet=None,labels=None,fold='[1]'):
        self.config = conf
        self.isSave = False
        self.isLoad = False
        self.foldInfo = fold
        self.labels = labels
        self.dao = RatingDAO(self.config, trainingSet, testSet)
        self.training = []
        self.trainingLabels = []
        self.test = []
        self.testLabels = []

    def readConfiguration(self):
        self.algorName = self.config['methodName']
        self.output = LineConfig(self.config['output.setup'])

    def printAlgorConfig(self):
        "show algorithm's configuration"
        print('Algorithm:'),self.config['methodName']
        print('Ratings dataSet:'),abspath(self.config['ratings'])
        if LineConfig(self.config['evaluation.setup']).contains('-testSet'):
            print('Test set:'),abspath(LineConfig(self.config['evaluation.setup']).getOption('-testSet'))
        print(self.dao.trainingSize())
        print(self.dao.testSize())
        print('='*80)

    def initModel(self):
        pass

    def buildModel(self):
        pass

    def saveModel(self):
        pass

    def loadModel(self):
        pass

    def predict(self):
        pass

    def execute(self):
        self.readConfiguration()
        if self.foldInfo == '[1]':
            self.printAlgorConfig()
        # load model from disk or build model
        if self.isLoad:
            self.loadModel()
        else:
            self.initModel()
            self.buildModel()

        # preict the ratings or item ranking
        print(self.foldInfo)
        prediction = self.predict()
        report = classification_report(self.testLabels, prediction, digits=4)
        # save model
        if self.isSave:
            print(self.foldInfo)
            self.saveModel()
        return report

In [ ]:
from sklearn.metrics import classification_report
import numpy as np
import os.path
from os.path import abspath     
from collections import defaultdict
from math import log,exp
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#CoDetector: Collaborative Shilling Detection Bridging Factorization and User Embedding
class CoDetector(SDetection):
    def __init__(self, conf, trainingSet=None, testSet=None, labels=None, fold='[1]'):
        super(CoDetector, self).__init__(conf, trainingSet, testSet, labels, fold)
        print(self.config)

    def readConfiguration(self):
        super(CoDetector, self).readConfiguration()
        extraSettings = LineConfig(self.config['CoDetector'])
        self.k = int(extraSettings['-k'])
        self.negCount = int(extraSettings['-negCount'])  # the number of negative samples
        if self.negCount < 1:
            self.negCount = 1

        self.regR = float(extraSettings['-gamma'])
        self.filter = int(extraSettings['-filter'])

        learningRate = LineConfig(self.config['learnRate'])
        self.lRate = float(learningRate['-init'])
        self.maxLRate = float(learningRate['-max'])
        self.maxIter = int(self.config['num.max.iter'])
        regular = LineConfig(self.config['reg.lambda'])
        self.regU, self.regI = float(regular['-u']), float(regular['-i'])

    def printAlgorConfig(self):
        super(CoDetector, self).printAlgorConfig()
        print(self.negCount)
        print(self.regR)
        print(self.filter)
        print('='* 80)

    def initModel(self):
        super(CoDetector, self).initModel()
        self.w = np.random.rand(len(self.dao.all_User)+1) / 20  # bias value of user
        self.c = np.random.rand(len(self.dao.all_User)+1)/ 20  # bias value of context
        self.G = np.random.rand(len(self.dao.all_User)+1, self.k) / 20  # context embedding
        self.P = np.random.rand(len(self.dao.all_User)+1, self.k) / 20  # latent user matrix
        self.Q = np.random.rand(len(self.dao.all_Item)+1, self.k) / 20  # latent item matrix

        # constructing SPPMI matrix
        self.SPPMI = defaultdict(dict)
        D = len(self.dao.user)
        print('Constructing SPPMI matrix...')
        # for larger data set has many items, the process will be time consuming
        occurrence = defaultdict(dict)
        for user1 in self.dao.all_User:
            iList1, rList1 = self.dao.allUserRated(user1)
            if len(iList1) < self.filter:
                continue
            for user2 in self.dao.all_User:
                if user1 == user2:
                    continue
                #if not occurrence[user1].has_key(user2):
                if not user2 in occurrence[user1]:
                    iList2, rList2 = self.dao.allUserRated(user2)
                    if len(iList2) < self.filter:
                        continue
                    count = len(set(iList1).intersection(set(iList2)))
                    if count > self.filter:
                        occurrence[user1][user2] = count
                        occurrence[user2][user1] = count
        maxVal = 0
        frequency = {}
        for user1 in occurrence:
            frequency[user1] = sum(occurrence[user1].values()) * 1.0
        D = sum(frequency.values()) * 1.0
        # maxx = -1
        for user1 in occurrence:
            for user2 in occurrence[user1]:
                try:
                    val = max([log(occurrence[user1][user2] * D / (frequency[user1] * frequency[user2]), 2) - log(
                        self.negCount, 2), 0])
                except ValueError:
                    print(self.SPPMI[user1][user2])
                    print(self.SPPMI[user1][user2] * D / (frequency[user1] * frequency[user2]))
                if val > 0:
                    if maxVal < val:
                        maxVal = val
                    self.SPPMI[user1][user2] = val
                    self.SPPMI[user2][user1] = self.SPPMI[user1][user2]

        # normalize
        for user1 in self.SPPMI:
            for user2 in self.SPPMI[user1]:
                self.SPPMI[user1][user2] = self.SPPMI[user1][user2] / maxVal

    def buildModel(self):
        # Jointly decompose R(ratings) and SPPMI with shared user latent factors P
        iteration = 0
        while iteration < self.maxIter:
            self.loss = 0

            self.dao.ratings = dict(self.dao.trainingSet_u, **self.dao.testSet_u)
            for user in self.dao.ratings:
                for item in self.dao.ratings[user]:
                    rating = self.dao.ratings[user][item]
                    error = rating - self.predictRating(user,item)
                    u = self.dao.all_User[user]
                    i = self.dao.all_Item[item]
                    p = self.P[u]
                    q = self.Q[i]
                    self.loss += error ** 2
                    # update latent vectors
                    self.P[u] += self.lRate * (error * q - self.regU * p)
                    self.Q[i] += self.lRate * (error * p - self.regI * q)

            for user in self.SPPMI:
                u = self.dao.all_User[user]
                p = self.P[u]
                for context in self.SPPMI[user]:
                    v = self.dao.all_User[context]
                    m = self.SPPMI[user][context]
                    g = self.G[v]
                    diff = (m - p.dot(g) - self.w[u] - self.c[v])
                    self.loss += diff ** 2
                    # update latent vectors
                    self.P[u] += self.lRate * diff * g
                    self.G[v] += self.lRate * diff * p
                    self.w[u] += self.lRate * diff
                    self.c[v] += self.lRate * diff
            self.loss += self.regU * (self.P * self.P).sum() + self.regI * (self.Q * self.Q).sum()  + self.regR * (self.G * self.G).sum()
            iteration += 1
            print('iteration:'),iteration

        # preparing examples
        self.training = []
        self.trainingLabels = []
        self.test = []
        self.testLabels = []

        for user in self.dao.trainingSet_u:
            self.training.append(self.P[self.dao.all_User[user]])
            self.trainingLabels.append(self.labels[str(float(user))])
        for user in self.dao.testSet_u:
            self.test.append(self.P[self.dao.all_User[user]])
            self.testLabels.append(self.labels[str(float(user))])

    def predictRating(self,user,item):
        u = self.dao.all_User[user]
        i = self.dao.all_Item[item]
        return self.P[u].dot(self.Q[i])

    def predict(self):
        classifier =  DecisionTreeClassifier(criterion='entropy')
        classifier.fit(self.training, self.trainingLabels)
        pred_labels = classifier.predict(self.test)
        print('Decision Tree:')
        return pred_labels

In [ ]:
import sys
from re import split
from multiprocessing import Process,Manager
from time import strftime,localtime,time
import re
import numpy as np
from os.path import abspath
class SDLib(object):
    def __init__(self,config):
        self.trainingData = []  # training data
        self.testData = []  # testData
        self.relation = []
        self.measure = []
        self.config =config
        self.ratingConfig =LineConfig(config['ratings.setup'])
        self.labels = FileIO.loadLabels(config['label'])

        if self.config.contains('evaluation.setup'):
            self.evaluation = LineConfig(config['evaluation.setup'])
            if self.evaluation.contains('-testSet'):
                #specify testSet
                self.trainingData = FileIO.loadDataSet(config, config['ratings'])
                self.testData = FileIO.loadDataSet(config, self.evaluation['-testSet'], bTest=True)
            elif self.evaluation.contains('-ap'):
                #auto partition
                self.trainingData = FileIO.loadDataSet(config,config['ratings'])
                self.trainingData,self.testData = DataSplit.\
                    dataSplit(self.trainingData,test_ratio=float(self.evaluation['-ap']))
            elif self.evaluation.contains('-cv'):
                #cross validation
                self.trainingData = FileIO.loadDataSet(config, config['ratings'])
                #self.trainingData,self.testData = DataSplit.crossValidation(self.trainingData,int(self.evaluation['-cv']))
        else:
            print('Evaluation is not well configured!')
            exit(-1)
            
        print('preprocessing...')


    def execute(self):
        #import the algorithm module
        if self.evaluation.contains('-cv'):
            k = int(self.evaluation['-cv'])
            if k <= 1 or k > 10:
                k = 3
            #create the manager used to communication in multiprocess
            manager = Manager()
            m = manager.dict()
            i = 1
            tasks = []
            for train,test in DataSplit.crossValidation(self.trainingData,k):
                fold = '['+str(i)+']'
                if self.config.contains('social'):
                    method = self.config['methodName'] + "(self.config,train,test,self.labels,self.relation,fold)"
                else:
                    method = self.config['methodName'] + "(self.config,train,test,self.labels,fold)"
               #create the process
                p = Process(target=run,args=(m,eval(method),i))
                tasks.append(p)
                i+=1
            #start the processes
            for p in tasks:
                p.start()
            #wait until all processes are completed
            for p in tasks:
                p.join()
            #compute the mean error of k-fold cross validation
            self.measure = [dict(m)[i] for i in range(1,k+1)]
            res = []
            pattern = re.compile('(\d+\.\d+)')
            countPattern = re.compile('\d+\\n')
            labelPattern = re.compile('\s\d{1}[^\.|\n|\d]')
            labels = re.findall(labelPattern, self.measure[0])
            values = np.array([0]*13,dtype=float)
            count = np.array([0]*5,dtype=int)
            for report in self.measure:
                values+= np.array([float(val) for val in re.findall(pattern,report)],dtype=float)
                count+=np.array(re.findall(countPattern,report),dtype=int)
            values/=k
            values=np.around(values,decimals=4)
            res.append('             precision  recall  f1-score  support\n\n')
            res.append('         '+labels[0]+'  '+'    '.join(np.array(values[0:3],dtype=str).tolist())+'   '+str(count[0])+'\n')
            res.append('         '+labels[1]+'  '+'    '.join(np.array(values[3:6],dtype=str).tolist())+'   '+str(count[1])+'\n\n')
            res.append('  avg/total   ' + '    '.join(np.array(values[6:9], dtype=str).tolist()) + '   ' + str(count[2]) + '\n')
            print('Total:')
            print(''.join(res))
            eval(method).execute()

def run(measure,algor,order):
    measure[order] = algor.execute()


In [ ]:
#import sys
#sys.path.append("..")
#from SDLib import SDLib
#from tool.config import Config

if __name__ == '__main__':

    print(('=')*80)
    print('SDLib: A Python library used to collect shilling detection methods.')
    print(('=')*80)
    print('Supervised Methods:')
    print('1. CoDetector')
    print(('-')*80)
    algor = -1
    conf = -1
    order = int(input('please enter the num of the method to run it:'))
    import time
    s = time.process_time()

    
    if order == 1:
        conf = Config('/content/drive/MyDrive/Attacks/ SD/SDLib-master/config/CoDetector.conf')

    elif order == 2:
        conf = Config('/content/drive/MyDrive/Attacks/ SD/SDLib-master/config/CoDetector of Average.conf')

    elif order == 3:
        conf = Config('/content/drive/MyDrive/Attacks/ SD/SDLib-master/config/CoDetector of Bandwagon.conf')

    elif order == 4:
        conf = Config('/content/drive/MyDrive/Attacks/ SD/SDLib-master/config/CoDetector of Love Hate.conf')

    else:
        print('Error num!')
        exit(-1)
    sd = SDLib(conf)
    sd.execute()
    e = time.process_time()
    #print("Run time: %f s") % (e - s)
    
    print(e - s)

SDLib: A Python library used to collect shilling detection methods.
Supervised Methods:
1. CoDetector
--------------------------------------------------------------------------------
please enter the num of the method to run it:2
loading training data...
preprocessing...
Algorithm:
Ratings dataSet:
(512, 8958, 132834)
(128, 8596, 36162)
256
1.0
4
Constructing SPPMI matrix...
Constructing SPPMI matrix...
Constructing SPPMI matrix...
Constructing SPPMI matrix...
Constructing SPPMI matrix...
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:
iteration:


KeyboardInterrupt: ignored